In [95]:
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import data_helpers
from hanziconv import HanziConv
import shutil

In [96]:
weibo_m_o = pd.read_csv('./data/Weibo_M.csv')
weibo_o = pd.read_csv('./data/Weibo.csv')
wechat_o = pd.read_csv('./data/Wechat.csv')
news_forum_o = pd.read_csv('./data/News_forum.csv')
dianping = pd.read_csv('./data/dianping135.csv')
#dianping = pd.read_csv('./data/dianping_full.csv')

In [97]:
# dianping_pos = dianping[dianping['rate']>4]
# dianping_pos['SENTIMENT'] = 'Positive'
# dianping_neg = dianping[dianping['rate']<2]
# dianping_neg['SENTIMENT'] = 'Negative'
# dianping_neu = dianping[dianping['rate']==3]
# dianping_neu['SENTIMENT'] = 'Neutral'
# dianping = pd.concat([dianping_pos,dianping_neg,dianping_neu])

In [98]:
# dianping = dianping[['content','SENTIMENT']].dropna(axis=0, how = 'any')
# dianping.columns = ['CONTENT','SENTIMENT']
# dianping['CONTENT'] = dianping['CONTENT'].str.replace('\n', '')
# dianping = dianping.reset_index(drop=True)
# dianping.to_csv('./data/dianping.csv',encoding = 'utf-8',index=False)
# dianping = pd.read_csv('./data/dianping.csv')

In [99]:
dianping = dianping.reset_index(drop=True)
shuffle_indices = np.random.permutation(np.arange(len(dianping)))
dianping = dianping.loc[shuffle_indices]
dianping = pd.concat([dianping[dianping.SENTIMENT == 'Positive'][:10000],
                      dianping[dianping.SENTIMENT == 'Negative'][:10000],
                      dianping[dianping.SENTIMENT == 'Neutral'][:10000]
                    ])

# Sentiment data

In [100]:
weibo_m_s = weibo_m_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
weibo_s = weibo_o[['COMMENT','SENTIMENT']].dropna(axis=0, how = 'any')
wechat_s = wechat_o[['CONTENT','SENTIMENT']].dropna(axis=0, how = 'any')

In [101]:
weibo_m_s.columns = ['CONTENT','SENTIMENT']
weibo_s.columns = ['CONTENT','SENTIMENT']

In [102]:
data = pd.concat([weibo_m_s,weibo_s,wechat_s])

In [103]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data = data.reset_index(drop=True)

In [104]:
length = []
for i in range(len(data)):
    data['CONTENT'][i] = HanziConv.toSimplified(data['CONTENT'][i]).encode('utf-8')
    length.append(len(data['CONTENT'][i]))
    if data['SENTIMENT'][i] == 'Irrelevant':
        data['SENTIMENT'][i] = 'Neutral'
length = np.array(length)

In [105]:
from sklearn.model_selection import train_test_split

dev, test = train_test_split(data, test_size=0.1, random_state = 100)

In [131]:
testing_folder = './data/processed/testing/'
if not os.path.exists(testing_folder):
    os.makedirs(testing_folder)
for the_file in os.listdir(testing_folder):
    file_path = os.path.join(testing_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
train_folder = './data/processed/training/'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)
for the_file in os.listdir(train_folder):
    file_path = os.path.join(train_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
        
dianping_folder = './data/processed/training/dianping/'
if not os.path.exists(dianping_folder):
    os.makedirs(dianping_folder)
for the_file in os.listdir(dianping_folder):
    file_path = os.path.join(dianping_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
snownlp_folder = './data/processed/training/snownlp/'
if not os.path.exists(snownlp_folder):
    os.makedirs(snownlp_folder)
for the_file in os.listdir(snownlp_folder):
    file_path = os.path.join(snownlp_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)
dev_folder = './data/processed/validation/'
if not os.path.exists(dev_folder):
    os.makedirs(dev_folder)
for the_file in os.listdir(dev_folder):
    file_path = os.path.join(dev_folder, the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
        #elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
        print(e)


In [132]:
# snownlp
shutil.copyfile("./data/pos.txt",snownlp_folder + "Positive.utf8")
shutil.copyfile("./data/neg.txt",snownlp_folder + "Negative.utf8")
os.mknod(snownlp_folder + "Neutral.utf8") 

In [134]:
# dianping
for x in dianping['SENTIMENT'].unique():
    np.savetxt(dianping_folder+ x.replace('/',' ') +'.utf8',\
                   dianping[dianping['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [135]:
# manulife
for x in dev['SENTIMENT'].unique():
    np.savetxt(dev_folder+ x.replace('/',' ') +'.utf8',\
                   train[train['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')
for x in test['SENTIMENT'].unique():
    np.savetxt(testing_folder+ x.replace('/',' ') +'.utf8',\
                   test[test['SENTIMENT'] == x]['CONTENT'].values, fmt='%s')

In [138]:
dev['SENTIMENT'].value_counts()

Neutral     13402
Positive      413
Negative      245
Name: SENTIMENT, dtype: int64

In [137]:
test['SENTIMENT'].value_counts()

Neutral     1469
Positive      52
Negative      42
Name: SENTIMENT, dtype: int64

In [16]:
from gensim.models import Word2Vec
w2vModel = Word2Vec.load('runs/trained_word2vec.model')

In [17]:
len(w2vModel.wv.vocab)

7355

In [68]:
from gensim.models import KeyedVectors
w2vModel = KeyedVectors.load_word2vec_format('runs/wiki.zh.vec')

In [19]:
vocab_obj = w2vModel.wv.vocab
word = u'保险'
print word
print vocab_obj[word].index

保险
2


In [41]:
dianping.head()

,CONTENT,SENTIMENT
0,仓储式购物环境，看着货架上一堆堆的物品，人的购物欲望就被激发起来了！进口食品很多，特别是西点...,Positive
1,有点远 在西安的时候逛得多我在岳麓区这边 每次去都要坐一个多小时的车 搞得后来没什么欲望去了...,Positive
2,实在要来赞一赞省博，首先网站做的很好，不多的能支持firefox的网站啊。网上订票系统非常靠...,Positive
3,第二次去了，正值周末，10点到已经只拿到下午2点的票子。估计排在后面的人是看不成了。马王堆是...,Positive
4,给予全五星的评价在于带我们的荣誉志愿者，唐老伯。呵呵，好像是财政厅退休公务员，带着湖南口音，...,Positive


In [70]:
from os import listdir

In [159]:
x = pd.read_html('./data/weibo sentiment/fei_jun_jian_e_yi_zhuang_ji_notations.xml')

ValueError: No tables found